# Using CDP Databases
Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

### Connecting to the database

CDP Seattle uses Firebase's 'Cloud Firestore' to store our data. However, a properly setup database host and associated database module _should_ have the same functionality.

**Note:** This notebook connects to the staging instance of Seattle's Firestore database. To use production data, connect to `cdp-seattle`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase

db = CloudFirestoreDatabase("stg-cdp-seattle")
db

<CloudFirestoreDatabase [stg-cdp-seattle]>

### Retrieving a single item
If you know the id of an item in a table, please use the `select_row_by_id` function provided.

In [2]:
event = db.select_row_by_id(table="event", id="1408bf08-d6c0-4ab7-96de-e1ef2294f075")
event

{'event_id': '1408bf08-d6c0-4ab7-96de-e1ef2294f075',
 'legistar_event_id': 4033,
 'event_datetime': datetime.datetime(2019, 7, 17, 9, 30),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_A_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_M_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Minutes.pdf',
 'video_uri': 'http://video.seattle.gov:8080/media/council/plan_071719_2511923V.mp4',
 'created': datetime.datetime(2019, 7, 21, 0, 29, 25, 1338),
 'body_id': 'ad388d06-06d4-4af5-a2f8-901e977cfaa7',
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=4033&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2018/2019-planning-land-use-and-zoning-committee?videoid=x105878'}

### Retrieving many items from a table

You may not know the id's of items you are looking for. In that case, use the `select_rows_as_list` function provided.

In [3]:
events = db.select_rows_as_list(table="event")
events[0]

{'event_id': '1408bf08-d6c0-4ab7-96de-e1ef2294f075',
 'video_uri': 'http://video.seattle.gov:8080/media/council/plan_071719_2511923V.mp4',
 'created': datetime.datetime(2019, 7, 21, 0, 29, 25, 1338),
 'body_id': 'ad388d06-06d4-4af5-a2f8-901e977cfaa7',
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=4033&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2018/2019-planning-land-use-and-zoning-committee?videoid=x105878',
 'legistar_event_id': 4033,
 'event_datetime': datetime.datetime(2019, 7, 17, 9, 30),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_A_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_M_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Minutes.pdf'}

### Joining with other tables

In the above event results, notice that a `body_id` is returned for each event. To attach body details to this we can use the python package `pandas` and query the `body` table. Let's first put each of the query results into `pandas.DataFrame` objects.

In [4]:
import pandas as pd

In [5]:
events = pd.DataFrame(events)
events.head()

,agenda_file_uri,body_id,created,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,http://legistar2.granicus.com/seattle/meetings...,ad388d06-06d4-4af5-a2f8-901e977cfaa7,2019-07-21 00:29:25.001338,2019-07-17 09:30:00,1408bf08-d6c0-4ab7-96de-e1ef2294f075,4033,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...
1,http://legistar2.granicus.com/seattle/meetings...,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.348645,2019-06-24 10:30:00,1d0c6214-1343-463d-b0ab-03a3d5e36c5d,4008,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...
2,http://legistar2.granicus.com/seattle/meetings...,b6792cc8-83cb-4c56-b38f-4997f16eb2ba,2019-07-21 00:16:04.078408,2019-06-25 14:00:00,276c6722-b35c-4341-b7fe-304d874a17ef,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...
3,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:21:31.105103,2019-06-27 12:00:00,294015e9-ee34-4de3-80d3-27175a127275,4013,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...
4,http://legistar2.granicus.com/seattle/meetings...,52aaf5d7-ed25-44c6-a236-d6ae915ff432,2019-07-20 23:28:27.305315,2019-07-15 10:30:00,2cbb7f4b-2e19-4f44-a99e-a0dcebdb3ac3,4029,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ar...


In [6]:
bodies = db.select_rows_as_list("body")
bodies = pd.DataFrame(bodies)
bodies.head()

,body_id,created,description,name
0,0239d692-4514-4d62-bdcb-3b0c470dee5d,2019-07-20 23:49:05.063092,None,Finance and Neighborhoods Committee
1,2448987c-36a0-48f3-a7ad-c76d20369d0c,2019-07-20 23:27:44.607321,None,"Civil Rights, Utilities, Economic Development,..."
2,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.123429,None,Select Committee on Homelessness and Housing A...
3,4364bde3-f1a4-47b7-9cf2-478296781681,2019-07-20 23:34:08.264936,None,"Civic Development, Public Assets, and Native C..."
4,52aaf5d7-ed25-44c6-a236-d6ae915ff432,2019-07-20 23:28:27.107855,None,Select Committee on Civic Arenas


In [7]:
expanded_event_details = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
expanded_event_details.head()

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
0,http://legistar2.granicus.com/seattle/meetings...,ad388d06-06d4-4af5-a2f8-901e977cfaa7,2019-07-21 00:29:25.001338,2019-07-17 09:30:00,1408bf08-d6c0-4ab7-96de-e1ef2294f075,4033,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...,2019-07-21 00:29:24.783820,None,"Planning, Land Use, and Zoning Committee"
1,http://legistar2.granicus.com/seattle/meetings...,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.348645,2019-06-24 10:30:00,1d0c6214-1343-463d-b0ab-03a3d5e36c5d,4008,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-20 23:44:41.123429,None,Select Committee on Homelessness and Housing A...
2,http://legistar2.granicus.com/seattle/meetings...,b6792cc8-83cb-4c56-b38f-4997f16eb2ba,2019-07-21 00:16:04.078408,2019-06-25 14:00:00,276c6722-b35c-4341-b7fe-304d874a17ef,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-21 00:16:03.860927,None,"Human Services, Equitable Development, and Ren..."
3,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:21:31.105103,2019-06-27 12:00:00,294015e9-ee34-4de3-80d3-27175a127275,4013,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-21 00:13:11.547208,None,"Housing, Health, Energy, and Workers’ Rights C..."
4,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:18:16.049660,2019-07-02 17:00:00,2cf94f65-6332-4b3b-9481-7e748671c54d,3992,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-21 00:13:11.547208,None,"Housing, Health, Energy, and Workers’ Rights C..."


`left_on` refers to the column name in the dataframe calling the operation.
In this case, the column to merge on is `body_id` in the events results.

Similarly, `right_on` refers to the column name in the dataframe to be passed to the operation.
In this case, the column to merge on is `id` in the bodies results.

`suffixes` is a tuple to use for adding suffixes to any columns with the same name between the two dataframes.
Commonly for CDP query results, these are columns such as `created`, which provide a `datetime` value for when that row was stored in the database.

Please refer to `pandas.DataFrame.merge` documentation for more details.

[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

### Filtering

You may notice that the function: `select_rows_as_list` allows for additional parameters to be passed: `filters`, `order_by`, and `limit`. Currently, it is very easy to filter or order by a single field for CDP instances that use a Cloud Firestore Database.

For example, let's request the events that occurred in the last week.

In [8]:
from datetime import datetime, timedelta
from cdptools.databases import WhereOperators
a_week_ago = datetime.utcnow() - timedelta(days=7)

# WhereOperators.gteq is short hand for "greater than or equal to"
recent_events = db.select_rows_as_list("event", filters=[("event_datetime", WhereOperators.gteq, a_week_ago)])
recent_events = pd.DataFrame(recent_events)
recent_events

,agenda_file_uri,body_id,created,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,http://legistar2.granicus.com/seattle/meetings...,66390eb4-69d5-49cb-a2f1-d538364f3b92,2019-07-20 23:52:29.762367,2019-07-16 12:00:00,9338fedb-ff60-4b8b-9467-dde1dd6a4afb,4032,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...
1,http://legistar2.granicus.com/seattle/meetings...,ad388d06-06d4-4af5-a2f8-901e977cfaa7,2019-07-21 00:29:25.001338,2019-07-17 09:30:00,1408bf08-d6c0-4ab7-96de-e1ef2294f075,4033,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...
2,http://legistar2.granicus.com/seattle/meetings...,4364bde3-f1a4-47b7-9cf2-478296781681,2019-07-20 23:34:08.446233,2019-07-17 14:00:00,a884f8be-20f5-46f2-a8ee-d93c1715b038,3998,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...
3,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:13:11.774135,2019-07-18 09:30:00,fc4c0164-4057-4701-8dc8-140b583a73eb,4034,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...


Unfortunately, for Cloud Firestore Databases, requesting to filter on multiple fields or requesting to filter and order by *may* result in an error but the error will provide  directions to add an index to the database to make that query possible. Please contact the maintainer of that CDP instance you are using and ask if they will add a composite index to the database.

Most operations can be done with simply queries, but in the case they can't, please refer to this [stackoverflow post](https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas) about filtering down dataframes that can be used while your CDP instance maintainer is adding an additional index.

Below is an example of a query failing due to too many filters or ordery by.

In [9]:
recent_events_missing_a_minutes_file = db.select_rows_as_list(
    "event",
    filters=[
        ("event_datetime", WhereOperators.gteq, a_week_ago),
        ("body_id", WhereOperators.eq, "66390eb4-69d5-49cb-a2f1-d538364f3b92")
    ]
)

FailedRequestError: Request failed. Response: [{'error': {'code': 400, 'message': 'The query requires an index. You can create it here: https://console.firebase.google.com/project/stg-cdp-seattle/database/firestore/indexes?create_composite=Ck1wcm9qZWN0cy9zdGctY2RwLXNlYXR0bGUvZGF0YWJhc2VzLyhkZWZhdWx0KS9jb2xsZWN0aW9uR3JvdXBzL2V2ZW50L2luZGV4ZXMvXxABGgsKB2JvZHlfaWQQARoSCg5ldmVudF9kYXRldGltZRABGgwKCF9fbmFtZV9fEAE', 'status': 'FAILED_PRECONDITION'}}]